In [1]:
import pandas as pd
import os


In [2]:
import os
import pandas as pd
folder_path = '/Users/jainivedhitha/Desktop/car_data'

city_dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):  
        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_excel(file_path, engine='openpyxl')

            city_name = filename.split('_')[0]
                       
            df['City'] = city_name
            
            city_dataframes.append(df)
            print(f"Successfully read {filename} and extracted city: {city_name}")
        except Exception as e:
            print(f"Error reading {filename}: {e}")

full_dataset = pd.concat(city_dataframes, ignore_index=True)


Successfully read jaipur_cars.xlsx and extracted city: jaipur
Successfully read bangalore_cars.xlsx and extracted city: bangalore
Successfully read kolkata_cars.xlsx and extracted city: kolkata
Successfully read chennai_cars.xlsx and extracted city: chennai
Successfully read hyderabad_cars.xlsx and extracted city: hyderabad
Successfully read delhi_cars.xlsx and extracted city: delhi


In [3]:
print(full_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8369 entries, 0 to 8368
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   new_car_detail    8369 non-null   object
 1   new_car_overview  8369 non-null   object
 2   new_car_feature   8369 non-null   object
 3   new_car_specs     8369 non-null   object
 4   car_links         8369 non-null   object
 5   City              8369 non-null   object
dtypes: object(6)
memory usage: 392.4+ KB
None


In [4]:
print(full_dataset.columns)


Index(['new_car_detail', 'new_car_overview', 'new_car_feature',
       'new_car_specs', 'car_links', 'City'],
      dtype='object')


In [5]:
import ast

In [6]:
def structure_data(df):
    d1_list = []
    d2_list = []
    d3_list = []
    d4_list = []
    d5_list = []
    for row in range(len(df)):
        a = df.iloc[row]["new_car_detail"]
        res = ast.literal_eval(a)
        d_df = pd.DataFrame([res])
        d1_list.append(d_df)
        
        
        b = df.iloc[row]["new_car_overview"]
        res = ast.literal_eval(b)
        o_df = pd.DataFrame([res])
        key_lst = []
        val_lst = []
        for i in o_df["top"]:
            for j in i:
                key_lst.append(j['key'])
                val_lst.append(j['value'])
        result = dict(zip(key_lst,val_lst))
        o_df = pd.DataFrame([result])
        d2_list.append(o_df)
        
        
        c = df.iloc[row]["new_car_feature"]
        res = ast.literal_eval(c)
        f_df = pd.DataFrame()
        for i in res.keys():
            if i == 'heading':
                f_df[res[i]] = [len(res["top"])]
            elif i == 'data':
                for a in res[i]:
                    for b in a:
                        if b == 'heading':
                            f_df[a[b]] = [len(a["list"])]
        d3_list.append(f_df)
        
        d = df.iloc[row]["new_car_specs"]
        res = ast.literal_eval(d)
        s_df = pd.DataFrame()
        for i in res.keys():
            if i == 'top':
                for j in res[i]:
                    s_df[j['key']] = [j['value']]
            elif i == 'data':
                for a in res[i]:
                    for b in a:
                        if b == 'heading':
                            s_df[a[b]] = [len(a["list"])]
                        elif b == 'list':
                            for j in a[b]:
                                s_df[j["key"]] = [j["value"]]
        s_df = s_df.rename(columns = {'Seats':'Seats_capacity'})
        d4_list.append(s_df)
        
        e = df.iloc[row]["car_links"]
        c_df = pd.DataFrame([e])
        d5_list.append(c_df)
    
    d1 = pd.concat(d1_list,ignore_index = True)
    d2 = pd.concat(d2_list,ignore_index = True)
    d3 = pd.concat(d3_list,ignore_index = True)
    d4 = pd.concat(d4_list,ignore_index = True)
    d5 = pd.concat(d5_list,ignore_index = True)
    
    
    df = d1.join(d2).join(d3).join(d4).join(d5)
    df.rename(columns = {0:"car_links"},inplace = True)
    
    return df

In [7]:
full_dataset = structure_data(full_dataset)

In [8]:
full_dataset.rename(columns={'bt': 'body type'}, inplace=True)


In [9]:
full_dataset.head()

,it,ft,body type,km,transmission,ownerNo,owner,oem,model,modelYear,...,Acceleration,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Gross Weight,BoreX Stroke,Compression Ratio,Ground Clearance Unladen,car_links
0,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,2015,...,11.9 Seconds,Tubeless,16,5,295-litres,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
1,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,2012,...,NaN,"Radial, Tubeless",NaN,5,268,1335,NaN,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
2,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,2016,...,15.05 Seconds,"Tubeless, Radial",NaN,5,235-litres,1250kg,73 X 82 mm,11.0:1,NaN,https://www.cardekho.com/used-car-details/used...
3,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,2017,...,12.9 Seconds,Tubeless,NaN,5,256,NaN,NaN,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
4,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,2016,...,18.6 Seconds,Tubeless Tyres,NaN,5,180-liters,1350kg,69 x 72 mm,NaN,NaN,https://www.cardekho.com/used-car-details/used...


In [10]:
full_dataset.columns

Index(['it', 'ft', 'body type', 'km', 'transmission', 'ownerNo', 'owner',
       'oem', 'model', 'modelYear', 'centralVariantId', 'variantName', 'price',
       'priceActual', 'priceSaving', 'priceFixedText', 'trendingText',
       'Registration Year', 'Insurance Validity', 'Fuel Type', 'Seats',
       'Kms Driven', 'RTO', 'Ownership', 'Engine Displacement', 'Transmission',
       'Year of Manufacture', 'Features', 'Comfort & Convenience', 'Interior',
       'Exterior', 'Safety', 'Entertainment & Communication', 'Mileage',
       'Engine', 'Max Power', 'Torque', 'Wheel Size', 'Seats_capacity',
       'Engine and Transmission', 'Color', 'Engine Type', 'Displacement',
       'Max Torque', 'No of Cylinder', 'Values per Cylinder',
       'Value Configuration', 'Fuel Suppy System', 'Turbo Charger',
       'Super Charger', 'Dimensions & Capacity', 'Length', 'Width', 'Height',
       'Wheel Base', 'Front Tread', 'Rear Tread', 'Kerb Weight',
       'Miscellaneous', 'Gear Box', 'Drive Type', 'S

In [11]:
full_dataset['Ground Clearance Unladen'].value_counts()

Ground Clearance Unladen
190mm     128
160        73
209 mm     39
180mm      35
120mm      32
178mm      32
185mm      32
155mm      28
192mm      24
145mm      16
205        12
170         9
116mm       9
210mm       8
184mm       6
200mm       6
140mm       5
110mm       4
91mm        3
170mm       3
210 mm      3
165         2
205mm       2
180         2
156mm       1
175mm       1
Name: count, dtype: int64

In [12]:
full_dataset['Engine'].value_counts()

Engine
1197 CC    1607
998 CC      640
1199 CC     489
1497 CC     451
1248 CC     424
           ... 
3993 CC       1
1948 CC       1
4806 CC       1
2994 CC       1
2979 CC       1
Name: count, Length: 150, dtype: int64

In [13]:
full_dataset = full_dataset.drop(columns = ['ft','Kms Driven','Seats','Seats_capacity','transmission','owner',
                                           'priceSaving','priceFixedText','trendingText','Engine Displacement'],index = 1 )

In [14]:
import re
import pandas as pd

def clean_registration_year(value):
    if pd.isna(value):
        return pd.NA
    year_matches = re.findall(r'\d{4}', str(value))
    
    if year_matches:

        return int(year_matches[0])
    else:
        return pd.NA 

full_dataset['Registration Year'] = full_dataset['Registration Year'].apply(clean_registration_year)

print(full_dataset['Registration Year'].head())


0    2015
2    2016
3    2017
4    2016
5    2007
Name: Registration Year, dtype: object


In [15]:
full_dataset['Insurance Validity'] = full_dataset['Insurance Validity'].str.replace(r'\binsurance\b', '', case=False, regex=True)
full_dataset['Insurance Validity'] = full_dataset['Insurance Validity'].str.strip()
print(full_dataset['Insurance Validity'].head())


0    Third Party
2    Third Party
3    Third Party
4    Third Party
5    Third Party
Name: Insurance Validity, dtype: object


In [16]:
full_dataset['Ownership'] = full_dataset['Ownership'].str.replace(r'\bOwner\b', '', case=False, regex=True)
full_dataset['Ownership'] = full_dataset['Ownership'].str.strip()
print(full_dataset['Ownership'].head())


0    Second
2    Second
3     First
4     First
5    Second
Name: Ownership, dtype: object


In [17]:
full_dataset['Engine'] = full_dataset['Engine'].str.replace(r'\bcc\b', '', case=False, regex=True)
full_dataset['Engine'] = full_dataset['Engine'].str.strip()
print(full_dataset['Engine'].head())

0    1396
2     998
3    1197
4     998
5    1493
Name: Engine, dtype: object


In [18]:
full_dataset['Mileage'] = full_dataset['Mileage'].str.replace(r'\bkmpl\b', '', case=False, regex=True)
full_dataset['Mileage'] = full_dataset['Mileage'].str.replace(r'\bkm/kg\b', '', case=False, regex=True)

full_dataset['Mileage'] = full_dataset['Mileage'].str.strip()
print(full_dataset['Mileage'].head())


0    22.54
2     23.1
3     18.9
4    20.51
5     16.8
Name: Mileage, dtype: object


In [19]:
#full_dataset['Max Power'] = full_dataset['Max Power'].str.replace('bhp', '', regex=False)
#full_dataset['Max Power'] = full_dataset['Max Power'].str.replace('@', '', regex=False)
#full_dataset['Max Power'] = full_dataset['Max Power'].str.replace('rpm', '', regex=False)
#full_dataset['Max Power'] = full_dataset['Max Power'].str.strip()
#print(full_dataset['Max Power'].head())

In [20]:
full_dataset['Torque'] = full_dataset['Torque'].str.replace('Nm', '', regex=False)
full_dataset['Torque'] = full_dataset['Torque'].str.replace('nm', '', regex=False)
full_dataset['Torque'] = full_dataset['Torque'].str.replace('kgm', '', regex=False)

full_dataset['Torque'] = full_dataset['Torque'].str.strip()
print(full_dataset['Torque'].head())


0     219.7
2        90
3    113.75
4        90
5        24
Name: Torque, dtype: object


In [21]:
full_dataset['Wheel Size'] = full_dataset['Wheel Size'].str.replace('R', '', regex=False)
full_dataset['Wheel Size'] = full_dataset['Wheel Size'].str.strip()
print(full_dataset['Wheel Size'].head())


0     16
2    NaN
3    NaN
4    NaN
5     14
Name: Wheel Size, dtype: object


In [22]:
full_dataset['price'] = full_dataset['price'].str.replace('Lakh', '', regex=False)
full_dataset['price'] = full_dataset['price'].str.replace('₹', '', regex=False)
full_dataset['price'] = full_dataset['price'].str.replace('Crore', '', regex=False)
full_dataset['price'] = full_dataset['price'].str.strip()
print(full_dataset['price'].head())

0    5.10
2    3.20
3    4.20
4    3.50
5    1.50
Name: price, dtype: object


In [23]:
full_dataset['km'] = full_dataset['km'].str.replace(',', '', regex=False)
full_dataset['km'] = full_dataset['km'].str.strip()
print(full_dataset['km'].head())


0    120000
2     80000
3     44392
4     40000
5    150000
Name: km, dtype: object


In [24]:
full_dataset['priceActual'] = full_dataset['priceActual'].str.replace('Lakh', '', regex=False)
full_dataset['priceActual'] = full_dataset['priceActual'].str.replace('₹', '', regex=False)
full_dataset['priceActual'] = full_dataset['priceActual'].str.replace('Crore', '', regex=False)
full_dataset['priceActual'] = full_dataset['priceActual'].str.strip()
print(full_dataset['priceActual'].head())

0    
2    
3    
4    
5    
Name: priceActual, dtype: object


In [45]:
full_dataset['Wheel Base'] = full_dataset['Wheel Base'].str.replace('mm','',regex = False)
full_dataset['Wheel Base'] = full_dataset['Wheel Base'].str.replace(',','',regex = False)
full_dataset['Wheel Base'] = full_dataset['Wheel Base'].str.replace('m','',regex = False)
full_dataset['Wheel Base'] = full_dataset['Wheel Base'].str.strip()
print(full_dataset['Wheel Base'].head())

0    2570
2    2425
3    2425
4    2400
5    2500
Name: Wheel Base, dtype: object


In [26]:
full_dataset['Front Tread'] = full_dataset['Front Tread'].str.replace('mm','',regex = False)
full_dataset['Front Tread'] = full_dataset['Front Tread'].str.replace(',','',regex = False)
full_dataset['Front Tread'] = full_dataset['Front Tread'].str.strip()
print(full_dataset['Front Tread'].head())

0    1505
2    1420
3    1479
4    1295
5     NaN
Name: Front Tread, dtype: object


In [48]:
full_dataset['Rear Tread'] = full_dataset['Rear Tread'].str.replace('mm','',regex = False)
full_dataset['Rear Tread'] = full_dataset['Rear Tread'].str.replace(',','',regex = False)
full_dataset['Rear Tread'] = full_dataset['Rear Tread'].str.replace('`','',regex = False)
full_dataset['Rear Tread'] = full_dataset['Rear Tread'].str.strip()
print(full_dataset['Rear Tread'].head())

0    1503
2    1410
3    1493
4    1290
5     NaN
Name: Rear Tread, dtype: object


In [52]:
full_dataset['Kerb Weight'] = full_dataset['Kerb Weight'].str.replace('kg','',regex = False)
full_dataset['Kerb Weight'] = full_dataset['Kerb Weight'].str.replace('875-905','875',regex = False)
full_dataset['Kerb Weight'] = full_dataset['Kerb Weight'].str.replace(',','',regex = False)
full_dataset['Kerb Weight'] = full_dataset['Kerb Weight'].str.strip()
print(full_dataset['Kerb Weight'].head())

0    1515
2     840
3    1100
4     890
5    1173
Name: Kerb Weight, dtype: object


In [29]:
full_dataset['Gear Box'] = full_dataset['Gear Box'].str.replace('Speed','',regex = False)
full_dataset['Gear Box'] = full_dataset['Gear Box'].str.replace('automatic','',regex = False)
full_dataset['Gear Box'] = full_dataset['Gear Box'].str.strip()
print(full_dataset['Gear Box'].head())

0    6
2    5
3    5
4    5
5    5
Name: Gear Box, dtype: object


In [62]:
full_dataset['Top Speed'] = full_dataset['Top Speed'].str.extract('(\d+)').astype(float)

print(full_dataset['Top Speed'].head())

0    180.0
2    150.0
3    165.0
4    152.0
5    185.0
Name: Top Speed, dtype: float64


In [70]:
full_dataset['Acceleration'] = full_dataset['Acceleration'].str.extract('(\d+)').astype(float)

print(full_dataset['Acceleration'].head())

0    11.0
2    15.0
3    12.0
4    18.0
5    11.0
Name: Acceleration, dtype: float64


In [32]:
full_dataset['Alloy Wheel Size'] = full_dataset['Alloy Wheel Size'].str.replace('R','',regex = False)
full_dataset['Alloy Wheel Size'] = full_dataset['Alloy Wheel Size'].str.strip()
print(full_dataset['Alloy Wheel Size'].head())

0     16
2    NaN
3    NaN
4    NaN
5     14
Name: Alloy Wheel Size, dtype: object


In [33]:
full_dataset['Cargo Volumn'] = full_dataset['Cargo Volumn'].str.extract('(\d+)').astype(float)

print(full_dataset['Cargo Volumn'].head())

0    295.0
2    235.0
3    256.0
4    180.0
5    352.0
Name: Cargo Volumn, dtype: float64


In [67]:
full_dataset['Gross Weight'] = full_dataset['Gross Weight'].str.extract('(\d+)').astype(float)

print(full_dataset['Gross Weight'].head())

0       NaN
2    1250.0
3       NaN
4    1350.0
5    1650.0
Name: Gross Weight, dtype: float64


In [35]:
full_dataset['BoreX Stroke'] = full_dataset['BoreX Stroke'].str.replace('mm','',regex = False)
full_dataset['BoreX Stroke'] = full_dataset['BoreX Stroke'].str.strip()
print(full_dataset['BoreX Stroke'].head())

0        NaN
2    73 X 82
3        NaN
4    69 x 72
5    75 x 87
Name: BoreX Stroke, dtype: object


In [36]:
full_dataset['Ground Clearance Unladen'] = full_dataset['Ground Clearance Unladen'].str.replace('mm','',regex = False)
full_dataset['Ground Clearance Unladen'] = full_dataset['Ground Clearance Unladen'].str.strip()
print(full_dataset['Ground Clearance Unladen'].head())

0    NaN
2    NaN
3    NaN
4    NaN
5    NaN
Name: Ground Clearance Unladen, dtype: object


In [37]:
full_dataset['Length'] = full_dataset['Length'].str.replace('mm','',regex = False)
full_dataset['Length'] = full_dataset['Length'].str.replace(',','',regex = False)
full_dataset['Length'] = full_dataset['Length'].str.strip()
print(full_dataset['Length'].head())

0    3985
2    3715
3    3765
4    3599
5    4310
Name: Length, dtype: object


In [38]:
full_dataset['Width'] = full_dataset['Width'].str.replace('mm','',regex = False)
full_dataset['Width'] = full_dataset['Width'].str.replace(',','',regex = False)
full_dataset['Width'] = full_dataset['Width'].str.strip()
print(full_dataset['Width'].head())

0    1734
2    1635
3    1660
4    1495
5    1695
Name: Width, dtype: object


In [39]:
full_dataset['Height'] = full_dataset['Length'].str.replace('mm','',regex = False)
full_dataset['Height'] = full_dataset['Length'].str.replace(',','',regex = False)
full_dataset['Height'] = full_dataset['Length'].str.strip()
print(full_dataset['Length'].head())

0    3985
2    3715
3    3765
4    3599
5    4310
Name: Length, dtype: object


In [74]:
full_dataset['Turning Radius'] = full_dataset['Turning Radius'].str.extract('(\d+)').astype(float)

print(full_dataset['Turning Radius'].head())

0    5.0
2    4.0
3    4.0
4    4.0
5    5.0
Name: Turning Radius, dtype: float64


In [75]:
full_dataset['Turning Radius'].value_counts()

Turning Radius
5.0       3454
4.0       3177
6.0        107
11.0        16
12.0         9
13.0         4
6250.0       1
Name: count, dtype: int64

In [41]:
full_dataset['Tyre Type'] = full_dataset['Tyre Type'].str.replace(',',' ',regex = False)
full_dataset['Tyre Type'] = full_dataset['Tyre Type'].str.strip()
print(full_dataset['Tyre Type'].head())

0            Tubeless
2    Tubeless  Radial
3            Tubeless
4      Tubeless Tyres
5     Tubeless Radial
Name: Tyre Type, dtype: object


In [76]:
full_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8368 entries, 0 to 8368
Data columns (total 67 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   it                             8368 non-null   int64  
 1   body type                      8368 non-null   object 
 2   km                             8368 non-null   object 
 3   ownerNo                        8368 non-null   int64  
 4   oem                            8368 non-null   object 
 5   model                          8368 non-null   object 
 6   modelYear                      8368 non-null   int64  
 7   centralVariantId               8368 non-null   int64  
 8   variantName                    8368 non-null   object 
 9   price                          8368 non-null   object 
 10  priceActual                    8368 non-null   object 
 11  Registration Year              8316 non-null   object 
 12  Insurance Validity             8364 non-null   object

In [77]:
full_dataset.head()

,it,body type,km,ownerNo,oem,model,modelYear,centralVariantId,variantName,price,...,Acceleration,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Gross Weight,BoreX Stroke,Compression Ratio,Ground Clearance Unladen,car_links
0,0,Hatchback,120000,2,Hyundai,Hyundai i20,2015,1487,Asta 1.4 CRDi,5.10,...,11.0,Tubeless,16,5,295.0,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
2,0,Hatchback,80000,2,Maruti,Maruti Celerio,2016,3982,VXI AT,3.20,...,15.0,Tubeless Radial,NaN,5,235.0,1250.0,73 X 82,11.0:1,NaN,https://www.cardekho.com/used-car-details/used...
3,0,Hatchback,44392,1,Hyundai,Hyundai Grand i10,2017,5602,1.2 Kappa Magna BSIV,4.20,...,12.0,Tubeless,NaN,5,256.0,NaN,NaN,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
4,0,Hatchback,40000,1,Maruti,Maruti Wagon R,2016,4319,AMT VXI,3.50,...,18.0,Tubeless Tyres,NaN,5,180.0,1350.0,69 x 72,NaN,NaN,https://www.cardekho.com/used-car-details/used...
5,0,Sedan,150000,2,Hyundai,Hyundai Verna,2007,1796,CRDi,1.50,...,11.0,Tubeless Radial,14,4,352.0,1650.0,75 x 87,17.8:1,NaN,https://www.cardekho.com/used-car-details/used...


In [78]:
full_dataset.to_csv('/Users/jainivedhitha/Desktop/car_dekho.csv',index = False)